# Q6

In [1]:
import igraph as ig
import json

g = ig.Graph.Read(f = 'dataset/edge_weight', format = 'ncol', directed = False)
gcc = g.components().giant()
ig.summary(g)

name_to_disname_loc = {}

with open('dataset/san_francisco_censustracts.json', 'r') as f:
    data = json.loads(f.readline())
    features = data['features']
    for feature in features:
        coordinates = feature['geometry']['coordinates'][0][0]
        latitude = 0
        longitude = 0
        for coordinate in coordinates:
            latitude += coordinate[0]
            longitude += coordinate[1]
        latitude /= len(coordinates)
        longitude /= len(coordinates)
        name_to_disname_loc[feature['properties']['MOVEMENT_ID']] = (feature['properties']['DISPLAY_NAME'], latitude, longitude)

seq = gcc.vs()
for i in seq:
    i['display name'] = name_to_disname_loc[i['name']][0]
    i['location'] = (name_to_disname_loc[i['name']][1], name_to_disname_loc[i['name']][2])
ig.summary(gcc)



IGRAPH UNW- 1887 311812 -- 
+ attr: name (v), weight (e)
IGRAPH UNW- 1880 311802 -- 
+ attr: display name (v), location (v), name (v), weight (e)


# Q7

In [2]:
mst = gcc.spanning_tree(weights = gcc.es["weight"])
vertices = mst.vs()
ig.summary(mst)
print()
print('+' + 90 * '-' + '+')
for e in mst.es():
    st = e.tuple
    print(str(vertices[st[0]]['display name']) + ' -- ' + str(vertices[st[1]]['display name']))
    print('+' + 90 * '-' + '+')

IGRAPH UNW- 1880 1879 -- 
+ attr: display name (v), location (v), name (v), weight (e)

+------------------------------------------------------------------------------------------+
4200 Amargosa Drive, Antioch -- 2900 Roosevelt Lane, Antioch
+------------------------------------------------------------------------------------------+
0 Brookhaven Court, Fairmont, Pacifica -- 0 Longview Drive, Westlake, Daly City
+------------------------------------------------------------------------------------------+
600 Hobart Terrace, Santa Clara -- 3400 Mauricia Avenue, Santa Clara
+------------------------------------------------------------------------------------------+
29300 Lassen Street, Tennyson - Alquire, Hayward -- 30800 San Clemente Street, Hayward
+------------------------------------------------------------------------------------------+
200 Clearview Drive, Vallejo -- 1400 Magazine Street, Vallejo
+---------------------------------------------------------------------------------------

Diamond Cove Terrace, Bayview, San Francisco -- 0 Park Lane, Brisbane
+------------------------------------------------------------------------------------------+
2100 Fieldcrest Avenue, Fairfield -- 7600 Pleasants Valley Road, Vacaville
+------------------------------------------------------------------------------------------+
2100 Wallace Avenue, Aptos -- 200 Vía Novella, Aptos
+------------------------------------------------------------------------------------------+
800 Easton Avenue, San Bruno -- 600 San Bruno Avenue East, San Bruno
+------------------------------------------------------------------------------------------+
200 Naglee Avenue, Outer Mission, San Francisco -- 100 Farallones Street, Oceanview, San Francisco
+------------------------------------------------------------------------------------------+
300 Hale Street, Excelsior, San Francisco -- 100 Rutledge Street, Bernal Heights, San Francisco
+------------------------------------------------------------------------

400 Verducci Drive, Serramonte, Daly City -- 100 Serramonte Boulevard, Serramonte, Daly City
+------------------------------------------------------------------------------------------+
600 Perfidia Street, Modesto -- 1800 Crestmoor Court, Ceres
+------------------------------------------------------------------------------------------+
5400 Ridgewood Drive, Ardenwood, Fremont -- 4900 Hildasue Terrace, Ardenwood, Fremont
+------------------------------------------------------------------------------------------+
100 Pearl Street, Oakland Ave - Harrison St, Oakland -- 200 Park View Terrace, Adams Point, Oakland
+------------------------------------------------------------------------------------------+
500 Elder Avenue, Sand City -- Trail 91, Seaside
+------------------------------------------------------------------------------------------+
0 Napa Road, Sonoma -- 800 Knight Street, Sonoma
+------------------------------------------------------------------------------------------+
700 V

+------------------------------------------------------------------------------------------+
800 Roselawn Avenue, Modesto -- 100 South Washington Street, Downtown Modesto, Modesto
+------------------------------------------------------------------------------------------+
1500 3rd Avenue, East Lake, Oakland -- 600 East 18th Street, Clinton, Oakland
+------------------------------------------------------------------------------------------+
0 Travis Avenue, Travis Air Force Base, Fairfield -- 2300 White Drive, Fairfield
+------------------------------------------------------------------------------------------+
2300 Oregon Street, South Berkeley, Berkeley -- 3000 Colby Street, Elmwood, Berkeley
+------------------------------------------------------------------------------------------+
5200 Fleming Avenue, Maxwell Park, Oakland -- 5600 Holway Street, Oakland
+------------------------------------------------------------------------------------------+
6300 Colby Street, Upper Telegraph, O

# Q8

In [3]:
import random

max_index = len(gcc.vs()) - 1 # 1879
gcc_vertices = gcc.vs()
num_of_experience = 1000
used_combine = set()
i = 0

num_of_satisfy = 0
while i < num_of_experience:
    cur_combine = ()
    while True:
        indices = set()
        while len(indices) < 3:
            indices.add(random.randint(0,max_index))
        li = list(indices)
        li.sort()
        cur_combine = tuple(li)
        if not cur_combine in used_combine:
            used_combine.add(cur_combine)
            break
    
    edges = gcc.es.select(_within = cur_combine)
    if len(edges) < 3:
        continue
    num = 'experience ' + str(i)
    i += 1
    print(num.center(90, '='))
    count = 1
    weights = []
    for edge in edges:
        weights.append(edge['weight'])
        st = edge.tuple
        print('edge '+str(count)+': ' + str(gcc_vertices[st[0]]['display name']) + \
              ' -- ' + str(gcc_vertices[st[1]]['display name']))
        print('weight: ' + str(edge['weight']))
        count += 1
        print()
    if weights[0] + weights[1] > weights[2] and weights[2] + weights[1] > weights[0] and \
    weights[0] + weights[2] > weights[1]:
        print('satisfy the triangle inequality')
        num_of_satisfy += 1
    else:
        print('not satisfy the triangle inequality')
print(90 * '=')
print('satisfying rate: ' + str(num_of_satisfy / num_of_experience))

=======================================experience 0=======================================
edge 1: 3400 Beard Road, Northgate, Fremont -- 3300 Davis Street, Peralta Hacienda, Oakland
weight: 2047.745

edge 2: 500 Webster Street, Western Addition, San Francisco -- 3300 Davis Street, Peralta Hacienda, Oakland
weight: 2046.78

edge 3: 500 Webster Street, Western Addition, San Francisco -- 3400 Beard Road, Northgate, Fremont
weight: 2735.38

satisfy the triangle inequality
=======================================experience 1=======================================
edge 1: 1200 San Mateo Drive, Menlo Park -- 2000 Waverley Street, Old Palo Alto, Palo Alto
weight: 736.67

edge 2: 400 Verducci Drive, Serramonte, Daly City -- 2000 Waverley Street, Old Palo Alto, Palo Alto
weight: 2318.06

edge 3: 1200 San Mateo Drive, Menlo Park -- 400 Verducci Drive, Serramonte, Daly City
weight: 1929.0549999999998

satisfy the triangle inequality
=======================================experience 2==============

======================================experience 21=======================================
edge 1: 1500 3rd Avenue, East Lake, Oakland -- 1000 56th Street, Gaskill, Oakland
weight: 1025.845

edge 2: 6500 Outlook Avenue, Millsmont, Oakland -- 1000 56th Street, Gaskill, Oakland
weight: 1267.095

edge 3: 6500 Outlook Avenue, Millsmont, Oakland -- 1500 3rd Avenue, East Lake, Oakland
weight: 978.835

satisfy the triangle inequality
======================================experience 22=======================================
edge 1: 400 Piercy Road, Edenvale, San Jose -- 600 Hobart Terrace, Santa Clara
weight: 1713.905

edge 2: 600 Hobart Terrace, Santa Clara -- 2000 Harmil Way, Central San Jose, San Jose
weight: 1218.785

edge 3: 400 Piercy Road, Edenvale, San Jose -- 2000 Harmil Way, Central San Jose, San Jose
weight: 1165.105

satisfy the triangle inequality
======================================experience 23=======================================
edge 1: 4400 Telegraph Avenue, Temescal, Oak

======================================experience 40=======================================
edge 1: 800 Franklin Street, Fillmore District, San Francisco -- 600 28th Street, Noe Valley, San Francisco
weight: 1036.585

edge 2: 800 Franklin Street, Fillmore District, San Francisco -- 1300 Wright Loop, Presidio, San Francisco
weight: 866.405

edge 3: 1300 Wright Loop, Presidio, San Francisco -- 600 28th Street, Noe Valley, San Francisco
weight: 1397.71

satisfy the triangle inequality
======================================experience 41=======================================
edge 1: 35500 Ebro Court, Centerville, Fremont -- 0 Donna Way, Sequoyah, Oakland
weight: 1568.105

edge 2: 33200 Central Avenue, Union City -- 35500 Ebro Court, Centerville, Fremont
weight: 311.23

edge 3: 33200 Central Avenue, Union City -- 0 Donna Way, Sequoyah, Oakland
weight: 1355.395

satisfy the triangle inequality
======================================experience 42=======================================
edge 1: 2

======================================experience 61=======================================
edge 1: 15500 Tracy Street, San Lorenzo -- 0 Via Diego, San Lorenzo
weight: 143.135

edge 2: 15500 Tracy Street, San Lorenzo -- 6500 Outlook Avenue, Millsmont, Oakland
weight: 982.66

edge 3: 6500 Outlook Avenue, Millsmont, Oakland -- 0 Via Diego, San Lorenzo
weight: 1241.04

not satisfy the triangle inequality
======================================experience 62=======================================
edge 1: 3000 Los Prados Street, East San Mateo, San Mateo -- 33500 7th Street, Union City
weight: 1791.44

edge 2: 3000 Los Prados Street, East San Mateo, San Mateo -- 2200 Shade Tree Lane, North San Jose, San Jose
weight: 2206.635

edge 3: 2200 Shade Tree Lane, North San Jose, San Jose -- 33500 7th Street, Union City
weight: 1887.03

satisfy the triangle inequality
======================================experience 63=======================================
edge 1: 300 Annette Lane, Santa Clara, Haywar

======================================experience 83=======================================
edge 1: 200 Silverlake Drive, Lakewood, Sunnyvale -- 400 31st Avenue, Beresford Park, San Mateo
weight: 1842.31

edge 2: 8700 Mission Rock Street, South of Market, San Francisco -- 400 31st Avenue, Beresford Park, San Mateo
weight: 2076.885

edge 3: 200 Silverlake Drive, Lakewood, Sunnyvale -- 8700 Mission Rock Street, South of Market, San Francisco
weight: 2714.03

satisfy the triangle inequality
======================================experience 84=======================================
edge 1: 900 Sycamore Drive, Midtown, Palo Alto -- 2000 Essex Street, South Berkeley, Berkeley
weight: 3486.33

edge 2: 200 O'Farrell Street, Tenderloin, San Francisco -- 2000 Essex Street, South Berkeley, Berkeley
weight: 1708.625

edge 3: 900 Sycamore Drive, Midtown, Palo Alto -- 200 O'Farrell Street, Tenderloin, San Francisco
weight: 2845.48

satisfy the triangle inequality
======================================

...

======================================experience 896======================================
edge 1: Petrolite Street, Richmond -- 2500 Parker Street, Elmwood, Berkeley
weight: 1478.4850000000001

edge 2: 2900 Broderick Street, Cow Hollow, San Francisco -- 2500 Parker Street, Elmwood, Berkeley
weight: 2251.31

edge 3: 2900 Broderick Street, Cow Hollow, San Francisco -- Petrolite Street, Richmond
weight: 1750.72

satisfy the triangle inequality
======================================experience 897======================================
edge 1: 3900 Seven Trees Boulevard, South San Jose, San Jose -- Spring Street, Central San Jose, San Jose
weight: 1069.21

edge 2: Spring Street, Central San Jose, San Jose -- 4200 Chaboya Hills Court, Evergreen, San Jose
weight: 1370.0900000000001

edge 3: 3900 Seven Trees Boulevard, South San Jose, San Jose -- 4200 Chaboya Hills Court, Evergreen, San Jose
weight: 876.47

satisfy the triangle inequality
======================================experience 898===

======================================experience 915======================================
edge 1: 300 Marylinn Drive, Milpitas -- 2000 Commodore Drive, East San Jose, San Jose
weight: 889.585

edge 2: 300 Marylinn Drive, Milpitas -- 1700 Eisenhower Street, Shoreview, San Mateo
weight: 1877.79

edge 3: 1700 Eisenhower Street, Shoreview, San Mateo -- 2000 Commodore Drive, East San Jose, San Jose
weight: 2152.86

satisfy the triangle inequality
======================================experience 916======================================
edge 1: 100 Burns Court, Pleasant Hill -- 800 Orange Blossom Way, Danville
weight: 908.435

edge 2: 800 Orange Blossom Way, Danville -- 6100 Harwood Avenue, Rockridge, Oakland
weight: 1593.0500000000002

edge 3: 100 Burns Court, Pleasant Hill -- 6100 Harwood Avenue, Rockridge, Oakland
weight: 1240.115

satisfy the triangle inequality
======================================experience 917======================================
edge 1: 7800 Lilac Court, Cupertino

======================================experience 935======================================
edge 1: 1300 Franklin Street, Western Addition, San Francisco -- 0 Ryan Court, Stanford
weight: 3434.685

edge 2: 0 Ryan Court, Stanford -- 26200 Regal Avenue, Harder - Tennyson, Hayward
weight: 2267.0649999999996

edge 3: 1300 Franklin Street, Western Addition, San Francisco -- 26200 Regal Avenue, Harder - Tennyson, Hayward
weight: 2662.87

satisfy the triangle inequality
======================================experience 936======================================
edge 1: 0 Rose Orchard Way, North San Jose, San Jose -- 1500 De Leon Way, Livermore
weight: 2229.12

edge 2: 700 Via Aires, San Lorenzo -- 1500 De Leon Way, Livermore
weight: 2107.06

edge 3: 700 Via Aires, San Lorenzo -- 0 Rose Orchard Way, North San Jose, San Jose
weight: 1951.34

satisfy the triangle inequality
======================================experience 937======================================
edge 1: 2400 19th Avenue, Highland 

======================================experience 956======================================
edge 1: 0 Oak Gate Place, Pleasant Hill -- 7600 Brighton Drive, Dublin
weight: 1380.43

edge 2: 600 East 8th Street, Downtown Oakland, Oakland -- 0 Oak Gate Place, Pleasant Hill
weight: 1536.62

edge 3: 600 East 8th Street, Downtown Oakland, Oakland -- 7600 Brighton Drive, Dublin
weight: 1705.02

satisfy the triangle inequality
======================================experience 957======================================
edge 1: 2600 Alvarado Street, Floresta Gardens - Bradrick, San Leandro -- 4200 Leigh Avenue, Cambrian, San Jose
weight: 2545.57

edge 2: 1100 Folsom Street, South of Market, San Francisco -- 4200 Leigh Avenue, Cambrian, San Jose
weight: 4071.77

edge 3: 1100 Folsom Street, South of Market, San Francisco -- 2600 Alvarado Street, Floresta Gardens - Bradrick, San Leandro
weight: 1830.935

satisfy the triangle inequality
======================================experience 958===============

======================================experience 982======================================
edge 1: 700 Folsom Street, South of Market, San Francisco -- 200 16th Avenue, Richmond District, San Francisco
weight: 1575.6599999999999

edge 2: 600 Sutter Street, Lower Nob Hill, San Francisco -- 700 Folsom Street, South of Market, San Francisco
weight: 591.9749999999999

edge 3: 600 Sutter Street, Lower Nob Hill, San Francisco -- 200 16th Avenue, Richmond District, San Francisco
weight: 1167.885

satisfy the triangle inequality
======================================experience 983======================================
edge 1: 1000 Beech Street, East Palo Alto -- 3400 Beard Road, Northgate, Fremont
weight: 1183.8

edge 2: 3400 Beard Road, Northgate, Fremont -- 30600 Palomares Road, Castro Valley
weight: 1131.1100000000001

edge 3: 1000 Beech Street, East Palo Alto -- 30600 Palomares Road, Castro Valley
weight: 2342.98

not satisfy the triangle inequality
======================================ex

# Q9

In [8]:
import heapq

# get an Eulerian walk
def euler(g, v, vset):
    edges = g.es.select(_source_in = [v])
    if len(edges) == 0:
        return [v]
    else:
        walk = [v]
        awalk(g, walk, set(), vset)
        for i in range(len(walk) - 1):
            edges = g.es.select(_between = ([walk[i]], [walk[i + 1]]))
            g.delete_edges(edges[0])
        res = []
        for vertex in walk:
            nextwalk = euler(g, vertex, vset)
            for i in nextwalk:
                res.append(i)
        return res

def awalk(g, res, visited, vset):
    cur = res[-1]
    if len(res) > 1 and cur == res[0]:
        return True
    else:
        edges = g.es.select(_between = ([cur], vset))
        for edge in edges:
            if not edge in visited:
                visited.add(edge)
                tu = edge.tuple
                nex = -1
                if tu[0] == cur:
                    nex = tu[1]
                else:
                    nex = tu[0]
                res.append(nex)
                nexres = awalk(g, res, visited, vset)
                if nexres:
                    return True
                else:
                    res.pop()
                    visited.remove(edge)
        return False

# shortest path (Dijkstra algorithm, only when not finding weight in the original graph)
def shortest_path(graph, idx1, idx2):
    v_set = [i for i in range(len(graph.vs))]
    
    expanded = {} # data -> (priority, who generate)
    generated = {} # data -> (priority, who generate)
    # initialize
    heap = [[0, idx1]] # heap initial: (priority, data)
    generated[idx1] = [0, idx1]
    # expand & generate
    while len(heap) != 0:
        # expand
        cur = heapq.heappop(heap)
        cur_idx = cur[1]
        cur_wei = cur[0]
        expanded[cur_idx] = generated[cur_idx]
        if cur_idx == idx2:
            break
        # generate
        edges = graph.es.select(_between = ([cur_idx], v_set))
        for edge in edges:
            if edge.tuple[0] == cur_idx:
                gen_idx = edge.tuple[1]
            else:
                gen_idx = edge.tuple[0]
            new_wei = edge['weight'] + cur_wei
            if gen_idx not in expanded:
                if gen_idx not in generated:
                    generated[gen_idx] = [new_wei, cur_idx]
                    heapq.heappush(heap, [new_wei, gen_idx])
                elif new_wei < generated[gen_idx][0]:
                    heap.remove([generated[gen_idx][0], gen_idx])
                    heap.append([new_wei, gen_idx])
                    heapq.heapify(heap)
                    generated[gen_idx] = [new_wei, cur_idx]
    
    # create path from idx1 to idx2
    shortest_wei = expanded[idx2][0]
    path = list()
    cur = idx2
    path.append(idx2)
    while cur != idx1:
        cur = expanded[cur][1]
        path.append(cur)
    path.reverse()
    
    return path, shortest_wei



In [9]:
import sys
sys.setrecursionlimit(10000)  # set the maximum depth as 10000

# (b) Eulerian spanning graph
edgelist = []
for e in mst.es():
    edgelist.append(e.tuple)

mst_double = mst.as_undirected()
mst_double.add_edges(edgelist)

# (c) get an Eulerian walk
mst1 = mst_double.as_undirected()
euwalk = euler(mst1, 0, mst1.vs())

# for i in euwalk:
#     print(mst.vs()[i]['name'])

1938
700
1250
1946
977
1946
1250
1945
1250
700
2148
1940
549
133
549
1940
2148
1939
2408
132
2408
1939
1243
1251
1950
702
550
1660
875
1660
550
702
1950
1251
548
1235
1952
551
1252
715
1274
715
1252
551
128
1884
552
52
1886
2153
1888
2153
1886
2524
1887
1283
821
2523
2149
2150
353
1942
353
2522
1943
701
2402
2676
2151
2676
2402
701
1249
701
1943
2522
1248
174
1248
2522
353
2150
2149
2523
1856
272
1856
2652
1856
2523
1941
2523
821
1283
1887
2524
1886
52
552
1884
2670
1892
1890
1284
716
2594
2362
1269
2362
2594
716
1284
1890
1893
718
918
553
929
553
918
1895
1908
418
822
2374
1909
719
1287
1270
1287
1566
66
1564
220
2237
218
473
382
2592
326
2592
62
105
62
2665
63
219
1563
104
64
2395
2307
2395
65
889
1207
889
65
2395
64
2157
2533
2628
2533
2157
64
104
472
103
1555
1556
1551
216
290
470
393
470
1553
435
59
435
1553
470
290
216
1551
957
1106
1550
1484
1550
392
1103
1597
606
289
1548
289
606
607
2431
1587
1549
1104
1549
1587
2431
607
606
1597
1595
604
1102
1586
1102
604
1595
1597
605
888
8

471
1492
1105
1492
471
615
1552
615
1554
448
1554
217
1554
104
1563
219
63
2665
62
2592
1562
2355
2394
2354
2394
2355
1562
2592
382
325
382
473
218
2237
1565
324
1578
324
1565
2237
220
1564
66
1566
195
2396
1557
2396
195
1566
1287
719
1894
454
2489
454
1894
719
1909
2374
822
418
1908
1895
1286
1895
918
718
1893
1285
1893
1890
717
1889
2154
1889
1885
1254
1885
1889
717
1890
1892
2670
1884
128
551
1952
1235
351
1907
235
688
1905
396
1868
1891
1868
396
175
545
1904
545
175
396
1905
688
235
1907
1236
74
300
917
2407
176
2407
917
975
820
77
820
698
1928
1926
936
1220
697
1924
697
1220
936
1926
352
1926
1928
547
1927
2521
436
2173
1222
1935
1331
1934
2232
1934
2376
1934
1331
1935
1933
1935
824
1332
2596
1332
824
1329
1311
557
1936
725
1947
725
1936
2221
1936
557
1937
1949
1334
1335
2363
153
123
153
2363
1309
1917
722
2535
722
1918
419
1931
2677
1931
2172
1932
723
1932
2172
1931
419
1310
419
1918
722
1917
1309
930
380
1919
2145
1929
2146
1930
2146
1929
2145
1919
695
1237
695
1920
916
1921
192

In [11]:
# (d) a 2-approximate tour
path = []
visited = set()
for index in euwalk:
    if not index in visited:
        path.append(index)
        visited.add(index)

total_weight = 0
final_path = []
for i in range(len(path) - 1):
    index1 = path[i]
    index2 = path[i + 1]
    edges = gcc.es.select(_between = ([index1], [index2]))
    if len(edges) > 0:
        total_weight += edges[0]['weight']
        final_path.append(index1)
        final_path.append(index2)
    else: # if there are no valid edge in the original graph, use shortest path 
        path1, weight = shortest_path(gcc, index1, index2)
        total_weight += weight
#         total_weight += (gcc.shortest_paths_dijkstra([index1], [index2], weights = gcc.es()['weight']))[0][0]
        for p in path1:
            final_path.append(p)

print('total weight: ' + str(total_weight))

total weight: 468008.83999999985


In [17]:
# upper bound
upper_bound = total_weight / sum(mst.es['weight'])
print(upper_bound)

1.6750004956905697


In [48]:
# print path
print(mst.vs[final_path[0]]['display name'] + ' -->\n')
for i in range(len(final_path)):
    if i%2 != 0:
        print(mst.vs[final_path[i]]['display name'] + ' -->\n')

3300 Brodie Drive, South San Jose, San Jose -->

3700 McLaughlin Avenue, South San Jose, San Jose -->

3700 Carick Place Way, East San Jose, San Jose -->

1800 Loch Ness Way, Evergreen, San Jose -->

2200 Terrena Valley Drive, East San Jose, San Jose -->

5300 Canyon Hills Lane, East San Jose, San Jose -->

2900 McLaughlin Avenue, South San Jose, San Jose -->

700 Overland Way, South San Jose, San Jose -->

2600 Pacer Lane, South San Jose, San Jose -->

2100 South 10th Street, South San Jose, San Jose -->

2500 Loomis Drive, South San Jose, San Jose -->

1900 Crinan Drive, South San Jose, San Jose -->

1000 Glenfinnan Drive, South San Jose, San Jose -->

3100 Oakbridge Drive, East San Jose, San Jose -->

2200 Lanai Avenue, East San Jose, San Jose -->

1500 June Avenue, Alum Rock, San Jose -->

1800 Loyola Drive, East San Jose, San Jose -->

2100 Naida Avenue, Alum Rock, San Jose -->

2400 Arden Way, Cassell, San Jose -->

2200 Waverly Avenue, Alum Rock, San Jose -->

1300 Santee Drive,


600 Panorama Drive, Midtown Terrace, San Francisco -->

0 Lopez Avenue, West of Twin Peaks, San Francisco -->

100 West Portal Avenue, West of Twin Peaks, San Francisco -->

1900 16th Avenue, Inner Sunset, San Francisco -->

1800 25th Avenue, Sunset District, San Francisco -->

1900 Lawton Street, Sunset District, San Francisco -->

1400 32nd Avenue, Sunset District, San Francisco -->

1400 40th Avenue, Sunset District, San Francisco -->

1300 46th Avenue, Sunset District, San Francisco -->

1700 47th Avenue, Sunset District, San Francisco -->

1800 32nd Avenue, Sunset District, San Francisco -->

2200 Rivera Street, Sunset District, San Francisco -->

1300 22nd Avenue, Sunset District, San Francisco -->

1300 16th Avenue, Inner Sunset, San Francisco -->

1300 10th Avenue, Inner Sunset, San Francisco -->

400 Moraga Street, Sunset District, San Francisco -->

2200 24th Avenue, Sunset District, San Francisco -->

0 Avoca Alley, West of Twin Peaks, San Francisco -->

3800 19th Street, D

# Q10

In [27]:
print(len(final_path))
print(len(path))

3852
1880


In [44]:
with open('dataset/path.csv', 'w') as f:
    f.write('Longitude, Latitude, Line Group (Path ID), Order of Points \n')
    # first one node
    loc = gcc.vs[final_path[0]]['location']
    f.write(str(loc[0]) + ', ' + str(loc[1]) + ', ' + '1, 1 \n')
    order = 2
    for i in range(1, len(final_path) - 1):
        if i%2 != 0:
            location = gcc.vs[final_path[i]]['location']
            f.write(str(location[0]) + ', ' + str(location[1]) + ', ' + '1, ' + str(order) + '\n')
            order += 1
    # last one node
    loc = gcc.vs[final_path[len(final_path) - 1]]['location']
    f.write(str(loc[0]) + ', ' + str(loc[1]) + ', ' + '1, ' + str(order) + '\n')
    